In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt

In [ ]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
# ss = pd.read_csv("../input/tensorflow-great-barrier-reef/example_sample_submission.csv")

train['annotations'] = train['annotations'].apply(lambda x: eval(x)) # annotationsが文字列として入っているため、eval()
train['n_annotations'] = train['annotations'].apply(len)
train['list_annotations'] = train['annotations'].apply(lambda x: [list(_.values()) for _ in x]) # リスト型式にcast

train.shape, test.shape

- Kaggleのページ上でディスプレイされている画像の中でヒトデが多いやつを探している(いちいち全部ダウンロードしたくないので)

In [ ]:
max_annots = 0
max_id = "" 
video_id = 0
for video_frame in range(1001, 1020):
    image_id = f'{video_id}-{video_frame}'
    len_annots = train[train['image_id']==image_id]['n_annotations'].item()
    if len_annots > max_annots:
        max_annots = len_annots
        max_id = image_id

print(max_id)
print(max_annots)

- 可視化パクリ元
https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507

- image_id 0-1001か1-10001を使う

In [ ]:
import cv2
from PIL import Image
image1_path = '../input/tensorflow-great-barrier-reef/train_images/video_0/1001.jpg'
image2_path = '../input/tensorflow-great-barrier-reef/train_images/video_1/10001.jpg'

In [ ]:
sample_ids = ['0-1001', '1-10001']
sample_bboxes = []
for sample_id in sample_ids:
    _bboxes = train[train['image_id']==sample_id]['list_annotations'].item()
    sample_bboxes.append(_bboxes)
    
    
sample_imgs = [
    cv2.imread(image1_path),
    cv2.imread(image2_path)
]

sample_img = sample_imgs[0]
sample_bbox = sample_bboxes[0]

In [ ]:
# 可視化用関数
def draw_bboxes(img, bboxes):
    for i in range(len(bboxes)):
            box = bboxes[i]
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = x0 + int(box[2])
            y1 = y0 + int(box[3])
            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
    return img

def draw_bboxes_predict(img, bboxes, scores, bbclasses, confthre, classes_dict):
    img = draw_bboxes(img, bboxes)
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
            if score < confthre:
                continue
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (box[0], box[1] - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

def display_cv2img(cv2_img):
    out_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(out_img))

# display cv2 image with bbox

In [ ]:
out_image = draw_bboxes(sample_img, sample_bbox)
display_cv2img(out_image)    # Since we load image using OpenCV we have to convert it 

In [ ]:
sample_bboxes

- 推論時想定可視化サンプル

In [ ]:
# # Get predictions
# # bboxes, bbclasses, scores = yolox_inference(img, model, test_size)
# bboxes, bbclasses, scores = sample_bboxes, [0], [0.99]
# # Draw predictions
# out_image = draw_bboxes_predict(sample_img, bboxes, scores, bbclasses, 0.0, {0: 'hitode'})

# # Since we load image using OpenCV we have to convert it 
# out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
# display(Image.fromarray(out_image))

# Augmentation
- 参考: https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [ ]:
# 自作クラスの読み込み
import sys
sys.path.append('../src/preprocess')
from augmentation import ObjectDetectionAugmentation

- yamlファイルから読み込み

In [ ]:
import yaml
with open('../config/config_train.yml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
config['transforms_args']

In [ ]:
transformer = ObjectDetectionAugmentation(config['transforms_args'])

In [ ]:
%%time
# 一枚だけ変換

_img, _bboxes = transformer.transform(sample_img, sample_bbox)
display_cv2img(draw_bboxes(_img, _bboxes))

In [ ]:
%%time
#　リストを全て変換

image_list, bboxes_list = transformer.transform_data(sample_imgs, sample_bboxes)
for _i, _b in zip(image_list, bboxes_list):
    display_cv2img(draw_bboxes(_i, _b))

# 実行時間調査

In [ ]:
%%time
# 単純なループ

for _img, _bboxes in zip(sample_imgs, sample_bboxes):
    draw_bboxes(*transformer.transform(_img, _bboxes))

In [ ]:
%%time
# frompyfucによる変換

image_list, bboxes_list = np.frompyfunc(transformer.transform, 2, 2)(pd.Series(sample_imgs), pd.Series(sample_bboxes))

In [ ]:
# np.vectorize?
# np.arrayを引数に持つ関数のvectorize

- Augmentationテスト

In [ ]:
import albumentations as A

out_image.shape

transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', min_area=100, min_visibility=0.1, label_fields=[]))

transformed = transform(image=sample_img, bboxes=sample_bbox)
transformed_img = transformed['image']
transformed_bboxes = transformed['bboxes']

transformed_img = draw_bboxes(transformed_img, transformed_bboxes)
display_cv2img(transformed_img)